In [12]:
import pandas as pd
import numpy as np
import time
import math

In [8]:
df=pd.read_csv("data_clean.csv")

In [9]:
# Testing changes
# df1=df[0:10]
# col=["Total Raised","Employees","Revenue"]

In [10]:
#order_df is the copy of data which is used to calculate ranks for each of the columns passed in the colnames variable
#After calculating ranks a new column is added to the order_df with name as colname_rank
#NA values are pushed to the bottom and assigned lowest rank (rank = 1); most relevant are the results with highest rank.

class relevance(object):
    
    def __init__(self, data, searchterm, weights):
        """
        Setting up global variables
        """
        self.starttime=time.time()
        self.order_df = data.copy()
        self.df_colnames = list(self.order_df)
        self.searchterm = searchterm
        self.rank_colname_list = []
        self.weights = weights
        #self.flag = False
        self.mostrelvdf = None
        
    def relevance_sort(self, colnames, popularitycolnames):
        
        #Ignoring search terms with character length less than 2
        if len(self.searchterm.strip())>2:
            #Segregating exact match results from the data to be relevantly ordered.
            #Checking the search term in Company Name and Ticker columns
            self.mostrelvdf = self.order_df[self.order_df['Company Name'].apply(
                lambda x: x.lower()).str.contains(self.searchterm.lower()) | self.order_df['Ticker'].apply(
                lambda x: str(x).lower()).str.contains(self.searchterm.lower())].reset_index()
            
            print(len(self.mostrelvdf))
            
            self.order_df = self.order_df[~self.order_df['Company Name'].apply(
                lambda x: x.lower()).str.contains(self.searchterm.lower())] 
            self.order_df = self.order_df[~self.order_df['Ticker'].apply(
                lambda x: str(x).lower()).str.contains(self.searchterm.lower())].reset_index()
            print(len(self.order_df))
            
        #Calculating ranks for all the columns
        self.order_df, self.rank_colname_list = self.calculate_rank(self.order_df, colnames, False)
        
        #Calculating one rank for all the popularity based columns; as we could accept only one weight
        #self.flag = True
        self.order_df, popularity_colName = self.calculate_rank(self.order_df, popularitycolnames, True)
        #self.flag = False
        self.rank_colname_list.append(popularity_colName)
        
        #Multiplying individual columns with their weights
        self.order_df[self.rank_colname_list] = self.order_df[self.rank_colname_list]*self.weights
        
        #Calculating mean rank and adding a order column
        self.order_df.insert(0,'order',self.order_df[self.rank_colname_list].mean(skipna = True, axis=1) )
        
        #print(order_df) #to print order -- for reference
        print("Generated order for ", len(self.order_df), " rows in ", time.time()-self.starttime," seconds")
        
        #Sorting based on order column to get relevant results on top
        self.order_df = self.order_df.sort_values(by='order', ascending = False, na_position = 'last' )
        
        #Concating exact match df and order df
        if self.mostrelvdf is not None:
            self.final_df = self.mostrelvdf.append(self.order_df)[self.df_colnames]
            print(len(self.final_df))
            return(self.final_df)
        else:
            print(len(self.order_df))
            return(self.order_df)[self.df_colnames]

    def calculate_rank(self, order_df, colnames, flag):
        rank_colname_list = []
        for i in colnames:
            rank_colname_list.append(i+'_rank') 
            ranking_list = order_df[i].rank(na_option = 'bottom')

    # In the below logic we are assigning lowest rank to the missing values so that they appear at the last

            for j in range (len(order_df[i])):
                if math.isnan(order_df[i][j]):
                    ranking_list[j] = 1

            order_df[i+'_rank'] = ranking_list
            
        #Creating a mean rank of all the popularity columns, so that we accept only one weight for them
        if flag:
            order_df.insert(0,'popularity_rank',order_df[rank_colname_list].mean(skipna = True, axis=1) )
            return order_df, 'popularity_rank'
        else:
            return order_df, rank_colname_list

In [13]:
relevance_obj = relevance(df, "uberfan" , [0.8,0.1,0,0,0,0,0.1])

#Testing

#These are quantitative columns 
col=["Total Raised","Employees","Growth Rate","# Active Investors","Revenue","Growth Rate Change"]

#These are popularity based columns, a single rank is calculated oveall for these columns
popcol = [ "Social Growth Rate","Compete Growth Rate","Web Growth Rate", "Facebook Likes", "Twitter Followers"
          , "Facebook Likes Change"]

#col=["Total Raised"]
#sorted_df = relevance_sort(df1,col)
sorted_df = relevance_obj.relevance_sort(col, popcol)
(sorted_df)

1
48484
('Generated order for ', 48484, ' rows in ', 8.700999975204468, ' seconds')
48485


,Unnamed: 0,Company ID,Company Name,Company Former Name,Company Also Known As,PBId,Description,Primary Industry Sector,Primary Industry Group,Primary Industry Code,...,Facebook Likes,Facebook Likes Change,Facebook Likes % Change,Majestic Referring Domains,Majestic Referring Domains Change,Majestic Referring Domains % Change,Twitter Followers,Twitter Followers Change,Twitter Followers % Change,PitchBook Link
0,7356,96885-28,Uberfan,NaN,NaN,96885-28,Developer of a social media application for ba...,Information Technology,Software,Social/Platform Software,...,NaN,NaN,NaN,8.0,0.0,0.000000,220.0,0.0,0.000000,NaN
3213,3220,51136-75,Uber Technologies,UberCab,Uber,51136-75,Provider of a mobile application that connects...,Information Technology,Software,Social/Platform Software,...,8793860.0,149218.0,1.726133,49044.0,29.0,0.057756,719038.0,7293.0,1.024758,NaN
5289,5296,10695-52,Facebook (FB),The Facebook,FB,10695-52,Operator of a social-networking platform. The ...,Information Technology,Software,Social/Platform Software,...,183783908.0,327663.0,0.178605,48162.0,107.0,0.221221,14014244.0,1215.0,0.008669,NaN
5610,5617,10377-37,Tesla (TSLA),Tesla Motors,NaN,10377-37,Manufacturer of a branded line of electric car...,Consumer Products and Services (B2C),Transportation,Automotive,...,1929747.0,10877.0,0.566859,16023.0,271.0,1.720959,1245467.0,13030.0,1.057262,NaN
7232,7239,50894-65,Lyft,Zimride,NaN,50894-65,Provider of an on-demand ride-sharing platform...,Information Technology,Software,Social/Platform Software,...,428564.0,10161.0,2.428587,12210.0,79.0,0.654160,206713.0,16931.0,8.921212,NaN
3572,3579,10359-55,Hulu,NaN,NaN,10359-55,Provider of an online video service that offer...,Consumer Products and Services (B2C),Media,"Broadcasting, Radio and Television",...,2517248.0,2763.0,0.109889,72464.0,11.0,0.015872,519502.0,17192.0,3.422441,NaN
3762,3769,61371-46,Magic Leap,NaN,NaN,61371-46,Developer of human computing interfaces and so...,Information Technology,Computer Hardware,"Computers, Parts and Peripherals",...,86338.0,72.0,0.083297,3798.0,-31.0,-0.798950,43463.0,110.0,0.253671,NaN
8515,8523,62181-28,WeWork,NaN,NaN,62181-28,Provider of office space designed to provide o...,Consumer Products and Services (B2C),Services (Non-Financial),Real Estate Services (B2C),...,360720.0,2608.0,0.728183,5722.0,8.0,0.137503,60000.0,287.0,0.481229,NaN
2224,2231,42936-49,Klarna,Kreditor Europe,NaN,42936-49,Provider of a billing and electronic commerce ...,Information Technology,Software,Vertical Market Software,...,29071.0,1044.0,3.725980,3998.0,4.0,0.082252,6018.0,22.0,0.376026,NaN
12518,12526,10640-89,MetroPCS Wireless (PCS),"GWI, GWI PCS, General Wireless",MetroPCS Communications,10640-89,Provider of wireless broadband personal commun...,Information Technology,Communications and Networking,Wireless Service Providers,...,596726.0,819.0,0.137438,5376.0,-37.0,-0.685245,82544.0,93.0,0.113429,NaN


In [30]:
# Questions:
# 1. What do you do about categorical features
# 2. Missing values? default sorting?
# 3. Can the description be used? (jargon distance)
# 4. Multiple rows have the same average, secondary sort column?

# 5. Revenue column has only 8k values, 40k rows are empty
# 6. Not sure how to handle "Facebook Likes Change"
# 7. 

